In [ ]:
# default_exp core

# osob

> This is a python module providing communication abstractions for the OpenScience Observatories telescope.org service

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev import *

In [ ]:
#export
import logging
from requests import session
import configparser
from os.path import expanduser
import json
from fastcore.basics import patch

## Telescope class

This class orginizes all interaction with the service. It keeps all state and provides higher level functions like `login` or `get_user_requests`.

In [ ]:
#export
class Telescope :

    url='https://www.telescope.org/'
    cameratypes={
        'constellation':'1',
        'galaxy':       '2',
        'cluster':      '3',
        'planet':'5',
        'coast':'6',
        'pirate':'7',
    }

    REQUESTSTATUS_TEXTS={
        1: "New",
        2: "New, allocated",
        3: "Waiting",
        4: "In progress",
        5: "Reallocate",
        6: "Waiting again",
        7: "Complete on site",
        8: "Complete",
        9: "Hold",
        10: "Frozen",
        20: "Expired",
        21: "Expired w/CJobs",
        22: "Cancelled",
        23: "Cancelled w/CJobs",
        24: "Invalid",
        25: "Never rises",
        26: "Other error",
    }

    def __init__(self,user,passwd,cache='.cache/jobs'):
        self.s=None
        self.user=user
        self.passwd=passwd
        self.tout=60
        self.retry=15
        self.login()
        self.cache=cache

In [ ]:
#export
@patch
def login(self: Telescope):
    log = logging.getLogger(__name__)
    payload = {'action': 'login',
               'username': self.user,
               'password': self.passwd,
               'stayloggedin': 'true'}
    log.debug('Get session ...')
    self.s=session()
    log.debug('Logging in ...')
    self.s.post(self.url+'login.php', data=payload)

In [ ]:
#export
@patch
def logout(self: Telescope):
    if self.s is None :
        self.s.post(self.url+'logout.php')
        self.s=None
 

In [ ]:
#local
config = configparser.ConfigParser()
config.read(expanduser('~/.config/telescope.ini'))

['/home/jochym/.config/telescope.ini']

In [ ]:
#local
OSO=Telescope(config['telescope.org']['user'], 
              config['telescope.org']['password'])

In [ ]:
#export
@patch
def get_user_requests(self: Telescope, sort='rid', folder=1):
    '''
    Get all user requests from folder (Inbox=1 by default),
    sorted by sort column ('rid' by default). 
    Possible sort columns are: 'rid', 'object', 'completion'
    The data is returned as a list of dictionaries.
    '''

    #fetch first batch        
    params={
        'limit': 100,
        'sort': sort,
        'folderid': folder}

    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "1-get-list-own",
                                               'params' : json.dumps(params)})
    res=[]
    dat=json.loads(rq.content)
    total=int(dat['data']['totalRequests'])
    res+=dat['data']['requests']

    # Fetch the rest
    params['limit']=total-len(res)
    params['startAfterRow']=len(res)
    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "1-get-list-own",
                                               'params' : json.dumps(params)})

    dat=json.loads(rq.content)
    total=int(dat['data']['totalRequests'])
    res+=dat['data']['requests']
    return res

In [ ]:
#local
reqlst=OSO.get_user_requests(sort='completion')
print(f'Number of users requests: {len(reqlst)}')
reqlst[:2]

Number of users requests: 1162


[{'id': '685130',
  'seen': '0',
  'usercomments': 'Mira',
  'objecttype': 'RADEC',
  'objectid': '19:19:02.60 +41:06:34.24',
  'objectname': 'EQ Lyr',
  'requesttime': '1589275435',
  'status': '8',
  'row': '1'},
 {'id': '685129',
  'seen': '0',
  'usercomments': 'Mira',
  'objecttype': 'RADEC',
  'objectid': '20:00:03.02 +22:46:51.55',
  'objectname': 'DQ Vul',
  'requesttime': '1589275435',
  'status': '8',
  'row': '2'}]

In [ ]:
#export
@patch
def get_user_folders(self: Telescope):
    '''
    Get all user folders. Returns list of dictionaries.
    '''
    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "0-get-my-folders"})
    return json.loads(rq.content)['data']

In [ ]:
#local
OSO.get_user_folders()

[{'id': '1', 'creationtime': '0', 'name': 'Inbox', 'count': '1162'},
 {'id': '2', 'creationtime': '0', 'name': 'Favourites', 'count': None},
 {'id': '3', 'creationtime': '0', 'name': 'Archive', 'count': '447'},
 {'id': '4', 'creationtime': '0', 'name': 'Trash', 'count': '52'},
 {'id': '461',
  'creationtime': '1407254495',
  'name': 'Complete',
  'count': '13'}]

In [ ]:
#local
OSO.logout()